In [1]:
from diffusers import DDIMPipeline

In [3]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
# from pipeline_controlnet_freedom import StableDiffusionControlNetFreedomPipeline
import torch


# controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)


# https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace
# controlnet = ControlNetModel.from_pretrained("CrucibleAI/ControlNetMediaPipeFace", subfolder="diffusion_sd15")

# https://huggingface.co/lllyasviel/control_v11p_sd15_openpose
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_openpose", torch_dtype=torch.float16, cache_dir='/home/aiteam/tykim/hugging')
 
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16,
    cache_dir='/home/aiteam/tykim/hugging')

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [5]:
pipe.scheduler.config

FrozenDict([('num_train_timesteps', 1000),
            ('beta_start', 0.00085),
            ('beta_end', 0.012),
            ('beta_schedule', 'scaled_linear'),
            ('trained_betas', None),
            ('skip_prk_steps', True),
            ('set_alpha_to_one', False),
            ('prediction_type', 'epsilon'),
            ('timestep_spacing', 'leading'),
            ('steps_offset', 1),
            ('_use_default_values', ['prediction_type', 'timestep_spacing']),
            ('_class_name', 'PNDMScheduler'),
            ('_diffusers_version', '0.6.0'),
            ('clip_sample', False)])

In [8]:
from diffusers import DDIMScheduler

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

In [10]:
len(pipe.scheduler.alphas_cumprod)

1000

In [21]:
pipe.scheduler.num_inference_steps

In [18]:
pipe.scheduler.config.num_train_timesteps // 20 

50

In [29]:
prev_timestep = 20 - pipe.scheduler.config.num_train_timesteps // 20
prev_timestep

-30

In [22]:
prev_timestep = 10 - self.config.num_train_timesteps // self.num_inference_steps

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prev_timestep = 10 - self.config.num_train_timesteps // self.num_inference_steps             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'self' is not defined

In [14]:
alphas_cumprod_prev = torch.cat(
            [torch.ones(1), pipe.scheduler.alphas_cumprod[:-1]], dim=0
        )

In [39]:
pipe.scheduler.alphas_cumprod.requires_grad

False

In [41]:
torch.cat([torch.ones(1), pipe.scheduler.alphas_cumprod[:-1]], dim=0).requires_grad

False

In [47]:
alphas = pipe.scheduler.alphas_cumprod
alphas_prev = torch.cat([torch.ones(1), pipe.scheduler.alphas_cumprod[:-1]], dim=0)

In [42]:
ddim_eta = 0.0

In [49]:
sigmas = ddim_eta * torch.sqrt((1 - alphas_prev) / (1 - alphas) * (1 - alphas / alphas_prev))

In [51]:
sigmas.requires_grad

False

In [52]:
sqrt_one_minus_alphas = np.sqrt(1. - pipe.scheduler.alphas_cumprod) 

In [53]:
sqrt_one_minus_alphas.requires_grad

False

In [ ]:
alphas = self.sacheduler.alphas_cumprod
alphas_prev = torch.cat([torch.ones(1), self.scheduler.alphas_cumprod[:-1]], dim=0)
sigmas = ddim_eta * torch.sqrt((1 - alphas_cumprod_prev) / (1 - alphas_cumprod) * (1 - alphas_cumprod / alphas_cumprod_prev))
sqrt_one_minus_alphas = np.sqrt(1. - self.scheduler.alphas_cumprod) 

In [36]:
np.sqrt(1. - pipe.scheduler.alphas_cumprod)

tensor([0.0292, 0.0413, 0.0506, 0.0585, 0.0655, 0.0718, 0.0777, 0.0831, 0.0883,
        0.0932, 0.0978, 0.1023, 0.1066, 0.1107, 0.1148, 0.1186, 0.1224, 0.1261,
        0.1297, 0.1333, 0.1367, 0.1401, 0.1434, 0.1466, 0.1498, 0.1529, 0.1560,
        0.1591, 0.1621, 0.1650, 0.1679, 0.1708, 0.1736, 0.1764, 0.1792, 0.1820,
        0.1847, 0.1873, 0.1900, 0.1926, 0.1952, 0.1978, 0.2004, 0.2029, 0.2054,
        0.2079, 0.2104, 0.2128, 0.2152, 0.2177, 0.2201, 0.2224, 0.2248, 0.2271,
        0.2295, 0.2318, 0.2341, 0.2364, 0.2387, 0.2409, 0.2432, 0.2454, 0.2476,
        0.2498, 0.2520, 0.2542, 0.2564, 0.2586, 0.2607, 0.2629, 0.2650, 0.2671,
        0.2692, 0.2713, 0.2734, 0.2755, 0.2776, 0.2797, 0.2817, 0.2838, 0.2858,
        0.2879, 0.2899, 0.2919, 0.2939, 0.2959, 0.2979, 0.2999, 0.3019, 0.3039,
        0.3059, 0.3078, 0.3098, 0.3117, 0.3137, 0.3156, 0.3176, 0.3195, 0.3214,
        0.3233, 0.3252, 0.3271, 0.3290, 0.3309, 0.3328, 0.3347, 0.3366, 0.3385,
        0.3403, 0.3422, 0.3440, 0.3459, 

In [34]:
import numpy as np

to_torch = lambda x: x.clone().detach().to(torch.float32)

to_torch(np.sqrt(1. - pipe.scheduler.alphas_cumprod.cpu()))

tensor([0.0292, 0.0413, 0.0506, 0.0585, 0.0655, 0.0718, 0.0777, 0.0831, 0.0883,
        0.0932, 0.0978, 0.1023, 0.1066, 0.1107, 0.1148, 0.1186, 0.1224, 0.1261,
        0.1297, 0.1333, 0.1367, 0.1401, 0.1434, 0.1466, 0.1498, 0.1529, 0.1560,
        0.1591, 0.1621, 0.1650, 0.1679, 0.1708, 0.1736, 0.1764, 0.1792, 0.1820,
        0.1847, 0.1873, 0.1900, 0.1926, 0.1952, 0.1978, 0.2004, 0.2029, 0.2054,
        0.2079, 0.2104, 0.2128, 0.2152, 0.2177, 0.2201, 0.2224, 0.2248, 0.2271,
        0.2295, 0.2318, 0.2341, 0.2364, 0.2387, 0.2409, 0.2432, 0.2454, 0.2476,
        0.2498, 0.2520, 0.2542, 0.2564, 0.2586, 0.2607, 0.2629, 0.2650, 0.2671,
        0.2692, 0.2713, 0.2734, 0.2755, 0.2776, 0.2797, 0.2817, 0.2838, 0.2858,
        0.2879, 0.2899, 0.2919, 0.2939, 0.2959, 0.2979, 0.2999, 0.3019, 0.3039,
        0.3059, 0.3078, 0.3098, 0.3117, 0.3137, 0.3156, 0.3176, 0.3195, 0.3214,
        0.3233, 0.3252, 0.3271, 0.3290, 0.3309, 0.3328, 0.3347, 0.3366, 0.3385,
        0.3403, 0.3422, 0.3440, 0.3459, 

In [12]:
torch.full((32, 1, 1, 1), pipe.scheduler.alphas_cumprod[500])

tensor([[[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]],


        [[[0.2763]]]])